# FINAL PROJECT: BOOTSTRAPPING

### Name: Carlos Cocha

#### Loading the required packages

In [1]:
import statistics
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

### 1. Read the data: x𝑖 ± 𝜎x𝑖, y𝑖 ± 𝜎y𝑖